In [1142]:
import input as i

matrix, accuracy = i.reader().read_from_file("tests/matrix3x3.txt")

In [1143]:
import numpy as np


def sort_diagonal_predominance(A: np.ndarray[np.float64]):
    indeces = []

    strict = False
    for row in A:
        if max(row) >= sum(row) - max(row):
            if max(row) > sum(row) - max(row):
                strict = True
            indeces.append(np.argmax(row))
        else:
            raise ValueError("impossible!")

    uniq = set(indeces)
    if len(uniq) != len(indeces) or not (strict):
        raise ValueError("impossible!")

    A = np.take(A, indeces, axis=1)

    return A

In [1144]:
try:
    A = sort_diagonal_predominance(matrix[:, :-1])
except:
    A = matrix[:, :-1]

In [1145]:
matrix[:, -1:]

array([[14.],
       [12.],
       [13.]])

In [1146]:
def make_C_and_D_matrix(A, B):
    matrix = np.hstack((A, B * -1))
    res = np.zeros((0, matrix.shape[1]))
    for index, row in enumerate(matrix):
        tmp = row / row[index]
        tmp[index] = 0
        res = np.vstack((res, tmp))
    res = res * -1
    print(res, "\n")
    return res[:, :-1], res[:, -1:]

In [1147]:
C, D = make_C_and_D_matrix(A, matrix[:, -1:])
print(C, "\n")
print(D)

[[-0.  -0.2 -0.2  1.4]
 [-0.1 -0.  -0.1  1.2]
 [-0.1 -0.2 -0.   1.3]] 

[[-0.  -0.2 -0.2]
 [-0.1 -0.  -0.1]
 [-0.1 -0.2 -0. ]] 

[[1.4]
 [1.2]
 [1.3]]


In [1148]:
def check_norma(A: np.ndarray[np.float64]):
    return norma(A) < 1


def norma(A: np.ndarray[np.float64]):
    return max([sum(abs(row)) for row in A])

In [1149]:
check_norma(C)

True

In [1150]:
def abs_deviations(k: np.ndarray[np.float64], prev: np.ndarray[np.float64]) -> int:
    return max(abs(k - prev))[0]


def discrepancy():
    return 0


def relative_differences():
    return 0

In [1151]:
def iterational_method(C, D: np.ndarray[np.float64], accuracy, max_iter):
    headers = ["x " + str(i) for i in range(C.shape[0], 0, -1)]
    headers.append("Критерий по абсолютным отклонениям")
    headers.append("Критерий по относительным разностям")
    headers.append("Критерий по невязке")

    accuracy_list = [0, 0, 0]
    result_matrix = [D]
    x = D
    for _ in range(max_iter - 1):
        x = C @ x + D
        criteria_abs = abs_deviations(x, result_matrix[-1])
        criteria_disc = discrepancy()
        criteria_rel = relative_differences()
        result_matrix.append(x)
        accuracy_list.append(criteria_abs)
        accuracy_list.append(criteria_disc)
        accuracy_list.append(criteria_rel)
        if (
            criteria_abs <= accuracy
            and criteria_disc <= accuracy
            and criteria_rel <= accuracy
        ):
            res = np.squeeze(np.array(result_matrix))
            result = np.hstack((res, np.array(accuracy_list).reshape(-1, 3)))
            return result, headers

    print("Достигли максимального кол-во итераций")

    res = np.squeeze(np.array(result_matrix))
    result = np.hstack((res, np.array(accuracy_list).reshape(-1, 3)))
    return result, headers


In [1152]:
import util

res, headers = iterational_method(C, D, 0.01                                                    , 1000)
                                    
util.print_table(res, headers)

     x 3       x 2      x 1    Критерий по абсолютным отклонениям    Критерий по относительным разностям    Критерий по невязке
--------  --------  -------  ------------------------------------  -------------------------------------  ---------------------
1.4       1.2       1.3                                  0                                             0                      0
0.9       0.93      0.92                                 0.5                                           0                      0
1.03      1.018     1.024                                0.13                                          0                      0
0.9916    0.9946    0.9934                               0.0384                                        0                      0
1.0024    1.0015    1.00192                              0.0108                                        0                      0
0.999316  0.999568  0.99946                              0.003084                                      0